In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

In [17]:
data = pd.read_csv('../MA_PREDICTOR/data/ma_data_car_clean.csv')

In [18]:
data.head()

,consideration_offered,shares_acquired,shares_at_announcement,acquiror_code,target_code,month,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target,car
0,Cash,full,no,50102030,50103030,1,cross_border,business_sector,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,0.006854
1,Other,full,no,54201030,63103010,1,cross_border,not_related,Consumer Non-Cyclicals,Personal & Household Products & Services,Academic & Educational Services,Academic & Educational Services,-0.010266
2,Other,full,no,57201030,57201020,1,cross_border,industry_group,Technology,Software & IT Services,Technology,Software & IT Services,0.007746
3,Cash,full,no,52102010,51101010,1,national,not_related,Industrials,Industrial Goods,Basic Materials,Chemicals,-0.011133
4,Cash,not_full,no,50102030,50102030,1,cross_border,industry,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,-0.003971


In [19]:
data.columns

Index(['consideration_offered', 'shares_acquired', 'shares_at_announcement',
       'acquiror_code', 'target_code', 'month', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
       'business_sector_target', 'car'],
      dtype='object')

In [20]:
y=data['car']

In [21]:
X=data[['consideration_offered', 'shares_acquired', 'shares_at_announcement', 'month', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
       'business_sector_target']]

# Splitting of the dataset

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

# Pipeline and GridSearch

In [23]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [24]:
cat_features=['consideration_offered', 'shares_acquired', 'shares_at_announcement', 'month', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
       'business_sector_target']

In [25]:
preprocessor = ColumnTransformer([
    ('cat_transformer', cat_transformer, cat_features)])

## ElasticNet

In [159]:
ElasticNet().get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'l1_ratio', 'max_iter', 'normalize', 'positive', 'precompute', 'random_state', 'selection', 'tol', 'warm_start'])

In [160]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessing', 'model', 'preprocessing__n_jobs', 'preprocessing__remainder', 'preprocessing__sparse_threshold', 'preprocessing__transformer_weights', 'preprocessing__transformers', 'preprocessing__verbose', 'preprocessing__cat_transformer', 'preprocessing__cat_transformer__categories', 'preprocessing__cat_transformer__drop', 'preprocessing__cat_transformer__dtype', 'preprocessing__cat_transformer__handle_unknown', 'preprocessing__cat_transformer__sparse', 'model__alpha', 'model__copy_X', 'model__fit_intercept', 'model__l1_ratio', 'model__max_iter', 'model__normalize', 'model__positive', 'model__precompute', 'model__random_state', 'model__selection', 'model__tol', 'model__warm_start'])

In [48]:
# Hyperparameter Grid
grid = {'model__alpha': [0.171, 0.174, 0.178, 0.182, 0.184, 0.186],
        'model__l1_ratio': [0,0.05, 0.1, 0.15, 0.2],
        'model__max_iter': [10000]}

In [50]:
# Combine preprocessor and linear model in pipeline
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', ElasticNet())])

In [51]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [52]:
# Fit data to Grid Search
search.fit(X_train,y_train);

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13.505158972351735, tolerance: 0.00270848662194968
  model = cd_fast.sparse_enet_coordinate_descent(


In [22]:
# Best score 2
search.best_score_

-0.002120279581575095

In [29]:
# Best score 3
search.best_score_

-0.0021202450752689714

In [36]:
# Best score 4
search.best_score_

-0.0021202445437064335

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.63233799683144, tolerance: 0.002134897697797967
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.814367203661002, tolerance: 0.002171952132502156
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.650380962976028, tolerance: 0.00213791916792

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.63036139424052, tolerance: 0.002134897697797967
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.812436694049278, tolerance: 0.002171952132502156
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.64885106887395, tolerance: 0.002137919167920

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.647165058182491, tolerance: 0.002137919167920729
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.95827228617846, tolerance: 0.0021995781105870666
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.90301437842222, tolerance: 0.00218936966125

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.899523901247267, tolerance: 0.0021893696612592245
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.901352422054181, tolerance: 0.0021893696612592245
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.634110890761725, tolerance: 0.00213489769

In [53]:
# Best score 5
search.best_score_

-0.0021202445437064335

In [54]:
# Best Params
search.best_params_

{'model__alpha': 0.184, 'model__l1_ratio': 0, 'model__max_iter': 10000}

In [153]:
# Best estimator
search.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat_transformer',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['consideration_offered',
                                                   'shares_acquired',
                                                   'shares_at_announcement',
                                                   'month', 'cross_border',
                                                   'relatedness',
                                                   'economic_sector_ac',
                                                   'business_sector_ac',
                                                   'economic_sector_target',
                                                   'business_sector_target'])])),
                ('model', ElasticNet(alpha=0.01, l1_ratio=0.2))])

In [155]:
search.cv_results_

{'mean_fit_time': array([0.0720283 , 0.06706514, 0.06978788, 0.0686924 , 0.07139521,
        0.07358255, 0.08356738, 0.07131333, 0.06183505]),
 'std_fit_time': array([0.00172139, 0.00199685, 0.00294417, 0.0028385 , 0.00449985,
        0.0043185 , 0.00913117, 0.00533084, 0.00985982]),
 'mean_score_time': array([0.02059455, 0.01920528, 0.02209353, 0.02080812, 0.01903114,
        0.02168961, 0.02188997, 0.01928802, 0.01591973]),
 'std_score_time': array([0.00167769, 0.00126214, 0.00490269, 0.00350674, 0.00200693,
        0.00337165, 0.00319695, 0.0007843 , 0.00541034]),
 'param_model__alpha': masked_array(data=[0.01, 0.01, 0.01, 0.1, 0.1, 0.1, 1, 1, 1],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_model__l1_ratio': masked_array(data=[0.2, 0.5, 0.8, 0.2, 0.5, 0.8, 0.2, 0.5, 0.8],
              mask=[False, False, False, False, False, False, False, False,
                  

For now, the best model is the following one:
    -ElasticNet with {'model__alpha': 0.184, 'model__l1_ratio': 0, 'model__max_iter': 10000}

## RandomForest

In [55]:
RandomForestRegressor().get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.823037600169421, tolerance: 0.002171952132502156
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.909977424148362, tolerance: 0.0021893696612592245
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.910393640038457, tolerance: 0.002189369661

In [26]:
# Hyperparameter Grid
grid={'model__bootstrap': [True, False],
      'model__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
      'model__max_features': ['auto', 'sqrt'],
      'model__min_samples_leaf': [1, 2, 4, 10],
      'model__min_samples_split': [2, 5, 10],
      'model__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [27]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor())])

In [28]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [29]:
# Fit data to Grid Search
search.fit(X_train,y_train);

KeyboardInterrupt: 

In [61]:
# Best score 1
search.best_score_

-0.0025065721534876877

In [ ]:
# Best score 2
search.best_score_

In [ ]:
# Best score 3
search.best_score_

In [62]:
# Best Params
search.best_params_

{'model__n_estimators': 150}